Making sure the data is properly being loaded

In [ ]:
from google.colab import drive
import os
import pandas as pd

# 1. PROPERLY MOUNT DRIVE
drive.flush_and_unmount()  # Clean previous mounts
drive.mount('/content/drive', force_remount=True)

# 2. VERIFY PATH EXISTS
data_dir = "/content/drive/MyDrive/tracedata"

# Check if directory exists
if not os.path.exists(data_dir):
    print(f"❌ Critical Error: Folder '{data_dir}' doesn't exist!")
    print("Do these EXACTLY:")
    print("1. Go to drive.google.com")
    print("2. Create folder 'tracedata' directly in 'My Drive' (NOT inside other folders)")
    print("3. Upload all CSV files to this folder")
    raise FileNotFoundError(f"Path {data_dir} not found")

# 3. CHECK FILES IN DIRECTORY
print("\n✅ Folder exists. Contents:")
files = os.listdir(data_dir)
print(f"Found {len(files)} files:")
print(*files[:5], sep="\n")  # Show first 5 files

# 4. LOAD DATA WITH ERROR HANDLING
def safe_load_csv(directory):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    if not csv_files:
        raise ValueError(f"No CSV files found in {directory}")

    print("\nLoading CSV files:")
    dfs = []
    for file in csv_files:
        file_path = os.path.join(directory, file)
        print(f"- Loading {file}")
        try:
            df = pd.read_csv(file_path)
            dfs.append(df)
        except Exception as e:
            print(f"⚠️ Error loading {file}: {str(e)}")

    if not dfs:
        raise ValueError("No CSV files could be loaded")

    return pd.concat(dfs, ignore_index=True)

# 5. EXECUTE WITH VERIFICATION
try:
    full_data = safe_load_csv(data_dir)
    print("\nSuccess! Data shape:", full_data.shape)
except Exception as e:
    print("\n❌ Loading failed:", str(e))

Mounted at /content/drive

✅ Folder exists. Contents:
Found 21 files:
1112_Sub01_walk_d10_1x1.csv
1112_Sub01_walk_d10_0x7.csv
1112_Sub01_walk_d10_0x9.csv
1112_Sub01_walk_d5_0x9.csv
1112_Sub01_walk_d5_0x7.csv

Loading CSV files:
- Loading 1112_Sub01_walk_d10_1x1.csv
- Loading 1112_Sub01_walk_d10_0x7.csv
- Loading 1112_Sub01_walk_d10_0x9.csv
- Loading 1112_Sub01_walk_d5_0x9.csv
- Loading 1112_Sub01_walk_d5_0x7.csv
- Loading 0117_Sub01_SA_i33_02.csv
- Loading 0117_Sub01_SD_d33_01.csv
- Loading 0117_Sub01_SD_d33_03.csv
- Loading 0117_Sub01_SA_i33_03.csv
- Loading 0117_Sub01_SD_d33_02.csv
- Loading 0117_Sub01_SA_i33_01.csv
- Loading 1112_Sub01_walk_i5_0x9.csv
- Loading 1112_Sub01_walk_i5_0x7.csv
- Loading 1112_Sub01_walk_i10_1x1.csv
- Loading 1112_Sub01_walk_i10_0x9.csv
- Loading 1112_Sub01_walk_i10_0x7.csv
- Loading 1112_Sub01_walk_d5_1x1.csv
- Loading 1112_Sub01_walk_i0_1x3.csv
- Loading 1112_Sub01_walk_i0_1x1.csv
- Loading 1112_Sub01_walk_i0_0x9.csv
- Loading 1112_Sub01_walk_i5_1x1.csv



KNN using the sklearn library - this is the most common library to use for the KNN model.

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from google.colab import drive

# Mount Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Custom Dataset Class
class GaitPredictionDataset(Dataset):
    def __init__(self, data, window_size=100, stride=20):
        self.window_size = window_size
        self.stride = stride

        # Features: Joint angles
        self.feature_cols = [
            'left_hip_angle',
            'left_knee_angle',
            'right_hip_angle',
            'right_knee_angle'
        ]

        # Targets: Prediction labels
        self.target_cols = [
            'locomotion_mode',
            'terrain_info',
            'gait_phase'
        ]

        # Validate columns
        missing_features = [col for col in self.feature_cols if col not in data.columns]
        missing_targets = [col for col in self.target_cols if col not in data.columns]

        if missing_features:
            raise ValueError(f"Missing feature columns: {missing_features}")
        if missing_targets:
            raise ValueError(f"Missing target columns: {missing_targets}")

        self.features = data[self.feature_cols].values.astype(np.float32)
        self.labels = data[self.target_cols].values
        self.samples = self._create_windows()

    def _create_windows(self):
        windows = []
        max_start = len(self.features) - self.window_size + 1
        for i in range(0, max_start, self.stride):
            window_features = self.features[i:i+self.window_size]
            window_labels = self.labels[i+self.window_size-1]  # Last label in window
            windows.append((window_features, window_labels))
        return windows

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, labels = self.samples[idx]
        return torch.FloatTensor(features), torch.FloatTensor(labels)

# Meta-Learning KNN System
class GaitKNNPredictor:
    def __init__(self):
        self.models = {
            'locomotion': KNeighborsClassifier(n_neighbors=5),
            'terrain': KNeighborsClassifier(n_neighbors=5),
            'gait_phase': KNeighborsClassifier(n_neighbors=5)
        }
        self.encoders = {
            'locomotion': LabelEncoder(),
            'terrain': LabelEncoder(),
            'gait_phase': LabelEncoder()
        }

    def train(self, X_train, y_locomotion, y_terrain, y_gait_phase):
        # Encode categorical labels
        y_locomotion_enc = self.encoders['locomotion'].fit_transform(y_locomotion)
        y_terrain_enc = self.encoders['terrain'].fit_transform(y_terrain)
        y_gait_enc = self.encoders['gait_phase'].fit_transform(y_gait_phase)

        print("Training locomotion model...")
        self.models['locomotion'].fit(X_train, y_locomotion_enc)

        print("Training terrain model...")
        self.models['terrain'].fit(X_train, y_terrain_enc)

        print("Training gait phase model...")
        self.models['gait_phase'].fit(X_train, y_gait_enc)

    def evaluate(self, X_test, y_locomotion, y_terrain, y_gait_phase):
        results = {}

        # Predict and decode labels
        for target in ['locomotion', 'terrain', 'gait_phase']:
            y_true = locals()[f"y_{target}"]
            y_pred_enc = self.models[target].predict(X_test)
            y_pred = self.encoders[target].inverse_transform(y_pred_enc)

            acc = accuracy_score(y_true, y_pred)
            report = classification_report(y_true, y_pred)
            results[target] = {'accuracy': acc, 'report': report}

        return results

# ===== MAIN EXECUTION =====
# Load data
data_dir = "/content/drive/MyDrive/tracedata"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

# Check data
print("Data columns:", full_data.columns.tolist())
print("\nSample data:")
print(full_data[['left_hip_angle', 'locomotion_mode', 'terrain_info', 'gait_phase']].head())

# Create dataset
dataset = GaitPredictionDataset(full_data, window_size=100, stride=20)
print(f"\nCreated {len(dataset)} sliding window samples")

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])
y_locomotion = np.array([y[0].item() for x, y in dataset])
y_terrain = np.array([y[1].item() for x, y in dataset])
y_gait = np.array([y[2].item() for x, y in dataset])

# Split data
X_train, X_test, y_lo_train, y_lo_test = train_test_split(X, y_locomotion, test_size=0.2, random_state=42)
_, _, y_tr_train, y_tr_test = train_test_split(X, y_terrain, test_size=0.2, random_state=42)
_, _, y_ga_train, y_ga_test = train_test_split(X, y_gait, test_size=0.2, random_state=42)

# Train and evaluate
predictor = GaitKNNPredictor()
predictor.train(X_train, y_lo_train, y_tr_train, y_ga_train)
results = predictor.evaluate(X_test, y_lo_test, y_tr_test, y_ga_test)

# Display results
print("\n=== Evaluation Results ===")
for target in results:
    print(f"\n{target.upper()} Model:")
    print(f"Accuracy: {results[target]['accuracy']:.2%}")
    print("Classification Report:")
    print(results[target]['report'])

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Data columns: ['locomotion_mode', 'terrain_info', 'walk_speed_info', 'gait_phase', 'gait_percentage', 'left_accel_x', 'left_accel_y', 'left_accel_z', 'left_gyro_x', 'left_gyro_y', 'left_gyro_z', 'right_accel_x', 'right_accel_y', 'right_accel_z', 'right_gyro_x', 'right_gyro_y', 'right_gyro_z', 'left_hip_angle', 'left_knee_angle', 'right_hip_angle', 'right_knee_angle', 'left_hip_angle_vel', 'left_knee_angle_vel', 'right_hip_angle_vel', 'right_knee_angle_vel']

Sample data:
   left_hip_angle  locomotion_mode  terrain_info  gait_phase
0           59.94              201             5           1
1           59.94              201             5           1
2           59.94              201             5           1
3           59.94              201             5           1
4           59.94              201             5           1

Created 9438 sliding window samples
Training locomotion model...
Training terra

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KNN using the Pytorch library, had to create the KNN using Pytorch's distance functions.

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Custom PyTorch Dataset
class GaitDataset(Dataset):
    def __init__(self, data, window_size=100, stride=20):
        self.window_size = window_size
        self.stride = stride

        # Features and targets
        self.feature_cols = [
            'left_hip_angle',
            'left_knee_angle',
            'right_hip_angle',
            'right_knee_angle'
        ]
        self.target_cols = [
            'locomotion_mode',
            'terrain_info',
            'gait_phase'
        ]

        # Validate columns
        missing_features = [col for col in self.feature_cols if col not in data.columns]
        missing_targets = [col for col in self.target_cols if col not in data.columns]
        if missing_features or missing_targets:
            raise ValueError(f"Missing columns: Features={missing_features}, Targets={missing_targets}")

        # Convert to numpy arrays
        self.features = data[self.feature_cols].values.astype(np.float32)
        self.labels = data[self.target_cols].values
        self.samples = self._create_windows()

    def _create_windows(self):
        windows = []
        max_start = len(self.features) - self.window_size + 1
        for i in range(0, max_start, self.stride):
            window_features = self.features[i:i+self.window_size]
            window_labels = self.labels[i+self.window_size-1]  # Last label in window
            windows.append((window_features, window_labels))
        return windows

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, labels = self.samples[idx]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long)

# Custom KNN Model in PyTorch
class PyTorchKNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            # Compute L2 distance (Euclidean)
            distances = torch.norm(self.X_train - x, dim=1)
            # Get k-nearest neighbors
            _, indices = torch.topk(distances, self.k, largest=False)
            # Vote for the most common label
            neighbor_labels = self.y_train[indices]
            unique_labels, counts = torch.unique(neighbor_labels, return_counts=True)
            pred_label = unique_labels[torch.argmax(counts)]
            predictions.append(pred_label)
        return torch.tensor(predictions)

# Meta-Learning KNN Trainer
class MetaKNN:
    def __init__(self):
        self.models = {
            'locomotion': PyTorchKNN(k=5),
            'terrain': PyTorchKNN(k=5),
            'gait_phase': PyTorchKNN(k=5)
        }
        self.encoders = {
            'locomotion': LabelEncoder(),
            'terrain': LabelEncoder(),
            'gait_phase': LabelEncoder()
        }

    def train(self, X_train, y_locomotion, y_terrain, y_gait_phase):
        # Encode categorical labels
        y_locomotion_enc = self.encoders['locomotion'].fit_transform(y_locomotion)
        y_terrain_enc = self.encoders['terrain'].fit_transform(y_terrain)
        y_gait_enc = self.encoders['gait_phase'].fit_transform(y_gait_phase)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_locomotion_tensor = torch.tensor(y_locomotion_enc, dtype=torch.long)
        y_terrain_tensor = torch.tensor(y_terrain_enc, dtype=torch.long)
        y_gait_tensor = torch.tensor(y_gait_enc, dtype=torch.long)

        # Train KNN models
        print("Training locomotion model...")
        self.models['locomotion'].fit(X_train_tensor, y_locomotion_tensor)

        print("Training terrain model...")
        self.models['terrain'].fit(X_train_tensor, y_terrain_tensor)

        print("Training gait phase model...")
        self.models['gait_phase'].fit(X_train_tensor, y_gait_tensor)

    def evaluate(self, X_test, y_locomotion, y_terrain, y_gait_phase):
        results = {}

        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        for target in ['locomotion', 'terrain', 'gait_phase']:
            y_true = locals()[f"y_{target}"]
            y_pred_enc = self.models[target].predict(X_test_tensor)
            y_pred = self.encoders[target].inverse_transform(y_pred_enc.numpy())

            acc = accuracy_score(y_true, y_pred)
            results[target] = acc

        return results

# ===== MAIN EXECUTION =====
# Load data
data_dir = "/content/drive/MyDrive/tracedata"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

# Create dataset
dataset = GaitDataset(full_data, window_size=100, stride=20)
print(f"Created {len(dataset)} sliding window samples")

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])
y_locomotion = np.array([y[0].item() for x, y in dataset])
y_terrain = np.array([y[1].item() for x, y in dataset])
y_gait = np.array([y[2].item() for x, y in dataset])

# Split data
X_train, X_test, y_lo_train, y_lo_test = train_test_split(X, y_locomotion, test_size=0.2, random_state=42)
_, _, y_tr_train, y_tr_test = train_test_split(X, y_terrain, test_size=0.2, random_state=42)
_, _, y_ga_train, y_ga_test = train_test_split(X, y_gait, test_size=0.2, random_state=42)

# Train and evaluate
meta_knn = MetaKNN()
meta_knn.train(X_train, y_lo_train, y_tr_train, y_ga_train)
results = meta_knn.evaluate(X_test, y_lo_test, y_tr_test, y_ga_test)

# Display results
print("\n=== Evaluation Results ===")
for target, acc in results.items():
    print(f"{target.capitalize()} Accuracy: {acc:.2%}")

Mounted at /content/drive
Created 9438 sliding window samples
Training locomotion model...
Training terrain model...
Training gait phase model...

=== Evaluation Results ===
Locomotion Accuracy: 99.79%
Terrain Accuracy: 99.58%
Gait_phase Accuracy: 90.41%


How this code works ->
This program classifies human movement patterns based on joint angle data using a meta-learning approach with K-Nearest Neighbors (KNN). It processes gait data to predict three aspects of movement: locomotion mode (e.g., walking, running), terrain type (e.g., flat ground, stairs), and gait phase (e.g., stance, swing). The data is first preprocessed into sliding windows, and three separate KNN models are trained for each classification task. Instead of traditional learning, KNN memorizes training samples and classifies new data points by finding the k-nearest neighbors and assigning the most common label. The meta-learning aspect comes from using multiple specialized KNN models, allowing the system to adapt to different movement classification tasks without interference.

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

class GaitDataset(Dataset):
    def __init__(self, data, window_size=100, stride=20):
        self.window_size = window_size
        self.stride = stride
        self.feature_cols = ['left_hip_angle', 'left_knee_angle', 'right_hip_angle', 'right_knee_angle']
        self.target_col = 'terrain_info'
        self.features = data[self.feature_cols].values.astype(np.float32)
        self.labels = data[self.target_col].values
        self.samples = self._create_windows()

    def _create_windows(self):
        windows = []
        for i in range(0, len(self.features) - self.window_size + 1, self.stride):
            windows.append((self.features[i:i+self.window_size], self.labels[i+self.window_size-1]))
        return windows

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, label = self.samples[idx]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

class PyTorchKNN:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            distances = torch.norm(self.X_train - x, dim=1)
            _, indices = torch.topk(distances, self.k, largest=False)
            neighbor_labels = self.y_train[indices]
            unique_labels, counts = torch.unique(neighbor_labels, return_counts=True)
            predictions.append(unique_labels[torch.argmax(counts)])
        return torch.tensor(predictions)

def train_and_evaluate_knn(X_train, X_test, y_train, y_test, accuracy_threshold=0.9):
    knn = PyTorchKNN(k=5)
    knn.fit(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    iterations = 0
    achieved_accuracy = 0.0
    while achieved_accuracy < accuracy_threshold:
        y_pred = knn.predict(torch.tensor(X_test, dtype=torch.float32)).numpy()
        achieved_accuracy = accuracy_score(y_test, y_pred)
        iterations += 1
        if iterations > 100:  # Prevent infinite loops
            break
    print(f"Iterations needed to reach {accuracy_threshold*100:.2f}% accuracy: {iterations}")
    print("Predicted terrain incline values:", y_pred)
    return achieved_accuracy

data_dir = "/content/drive/MyDrive/tracedata"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

dataset = GaitDataset(full_data, window_size=100, stride=20)
X = np.array([x.numpy().flatten() for x, y in dataset])
y = np.array([y.item() for x, y in dataset])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

final_accuracy = train_and_evaluate_knn(X_train, X_test, y_train, y_test)


Mounted at /content/drive
Iterations needed to reach 90.00% accuracy: 1
Predicted terrain incline values: [ 5 10 10 ...  0 10 10]


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load data
data_dir = "/content/drive/MyDrive/tracedata"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

import numpy as np
import torch

# Assuming dataset is already created and loaded
# dataset = [...]  # Ensure dataset is properly initialized

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])

# Check if y is a tuple/list (multiple targets) or a single tensor
if isinstance(dataset[0][1], (tuple, list)):
    y_locomotion = np.array([y[0].item() for x, y in dataset])
    y_terrain = np.array([y[1].item() for x, y in dataset])
    y_gait = np.array([y[2].item() for x, y in dataset])
else:
    y_terrain = np.array([y.item() for x, y in dataset])  # Single target case


# Check if y is a tuple/list (multiple targets) or a single tensor
if isinstance(dataset[0][1], (tuple, list)):
    y_locomotion = np.array([y[0].item() for x, y in dataset])
    y_terrain = np.array([y[1].item() for x, y in dataset])
    y_gait = np.array([y[2].item() for x, y in dataset])
else:
    y_terrain = np.array([y.item() for x, y in dataset])  # Single target case

# Create dataset
dataset = GaitDataset(full_data, window_size=100, stride=20)
print(f"Created {len(dataset)} sliding window samples")

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])

# Check if y is a tuple/list (multiple targets) or a single tensor
if isinstance(dataset[0][1], (tuple, list)):
    y_locomotion = np.array([y[0].item() for x, y in dataset])
    y_terrain = np.array([y[1].item() for x, y in dataset])
    y_gait = np.array([y[2].item() for x, y in dataset])
else:
    y_terrain = np.array([y.item() for x, y in dataset])  # Single target case


# Print unique class distribution for terrain incline
print("\n=== Terrain Incline Class Distribution ===")
unique_classes, class_counts = np.unique(y_terrain, return_counts=True)
for cls, count in zip(unique_classes, class_counts):
    print(f"Class {cls}: {count} samples")

# Ensure proper data splitting
X_train, X_test, y_tr_train, y_tr_test = train_test_split(X, y_terrain, test_size=0.2, random_state=42, shuffle=True)

# Train and evaluate
meta_knn = MetaKNN()
meta_knn.train(X_train, y_tr_train, y_tr_train, y_tr_train)  # Only training on terrain for debugging

# Predictions
y_pred_terrain = meta_knn.models['terrain'].predict(torch.tensor(X_test, dtype=torch.float32)).numpy()

# Print Classification Report
print("\n=== Classification Report for Terrain Incline ===")
print(classification_report(y_tr_test, y_pred_terrain))

# Confusion Matrix
cm = confusion_matrix(y_tr_test, y_pred_terrain)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=unique_classes, yticklabels=unique_classes)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Terrain Incline Prediction")
plt.show()


RuntimeError: a Tensor with 3 elements cannot be converted to Scalar

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Custom PyTorch Dataset
class GaitDataset(Dataset):
    def __init__(self, data, window_size=100, stride=20):
        self.window_size = window_size
        self.stride = stride

        # Features and targets
        self.feature_cols = [
            'left_hip_angle',
            'left_knee_angle',
            'right_hip_angle',
            'right_knee_angle'
        ]
        self.target_cols = [
            'locomotion_mode',
            'terrain_info',
            'gait_phase'
        ]

        # Validate columns
        missing_features = [col for col in self.feature_cols if col not in data.columns]
        missing_targets = [col for col in self.target_cols if col not in data.columns]
        if missing_features or missing_targets:
            raise ValueError(f"Missing columns: Features={missing_features}, Targets={missing_targets}")

        # Convert to numpy arrays
        self.features = data[self.feature_cols].values.astype(np.float32)
        self.labels = data[self.target_cols].values
        self.samples = self._create_windows()

    def _create_windows(self):
        windows = []
        max_start = len(self.features) - self.window_size + 1
        for i in range(0, max_start, self.stride):
            window_features = self.features[i:i+self.window_size]
            window_labels = self.labels[i+self.window_size-1]  # Last label in window
            windows.append((window_features, window_labels))
        return windows

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, labels = self.samples[idx]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long)

# Custom KNN Model in PyTorch
class PyTorchKNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            # Compute L2 distance (Euclidean)
            distances = torch.norm(self.X_train - x, dim=1)
            # Get k-nearest neighbors
            _, indices = torch.topk(distances, self.k, largest=False)
            # Vote for the most common label
            neighbor_labels = self.y_train[indices]
            unique_labels, counts = torch.unique(neighbor_labels, return_counts=True)
            pred_label = unique_labels[torch.argmax(counts)]
            predictions.append(pred_label)
        return torch.tensor(predictions)

# Meta-Learning KNN Trainer
class MetaKNN:
    def __init__(self):
        self.models = {
            'locomotion': PyTorchKNN(k=5),
            'terrain': PyTorchKNN(k=5),
            'gait_phase': PyTorchKNN(k=5)
        }
        self.encoders = {
            'locomotion': LabelEncoder(),
            'terrain': LabelEncoder(),
            'gait_phase': LabelEncoder()
        }

    def train(self, X_train, y_locomotion, y_terrain, y_gait_phase, max_epochs=100, target_accuracy=0.9):
        # Encode categorical labels
        y_locomotion_enc = self.encoders['locomotion'].fit_transform(y_locomotion)
        y_terrain_enc = self.encoders['terrain'].fit_transform(y_terrain)
        y_gait_enc = self.encoders['gait_phase'].fit_transform(y_gait_phase)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_locomotion_tensor = torch.tensor(y_locomotion_enc, dtype=torch.long)
        y_terrain_tensor = torch.tensor(y_terrain_enc, dtype=torch.long)
        y_gait_tensor = torch.tensor(y_gait_enc, dtype=torch.long)

        # Train KNN models
        for epoch in range(max_epochs):
            print(f"Epoch {epoch + 1}/{max_epochs}")

            print("Training locomotion model...")
            self.models['locomotion'].fit(X_train_tensor, y_locomotion_tensor)

            print("Training terrain model...")
            self.models['terrain'].fit(X_train_tensor, y_terrain_tensor)

            print("Training gait phase model...")
            self.models['gait_phase'].fit(X_train_tensor, y_gait_tensor)

            # Evaluate
            train_acc = self.evaluate(X_train, y_locomotion, y_terrain, y_gait_phase)
            print(f"Training Accuracy: {train_acc}")

            # Check for target accuracy
            if all(acc >= target_accuracy for acc in train_acc.values()):
                print(f"Target accuracy achieved in {epoch + 1} epochs")
                break

    def evaluate(self, X_test, y_locomotion, y_terrain, y_gait_phase):
        results = {}

        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        for target in ['locomotion', 'terrain', 'gait_phase']:
            y_true = locals()[f"y_{target}"]
            y_pred_enc = self.models[target].predict(X_test_tensor)
            y_pred = self.encoders[target].inverse_transform(y_pred_enc.numpy())

            acc = accuracy_score(y_true, y_pred)
            results[target] = acc

        return results

# ===== MAIN EXECUTION =====
# Load data
data_dir = "/content/drive/MyDrive/tracedata"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

# Create dataset
dataset = GaitDataset(full_data, window_size=100, stride=20)
print(f"Created {len(dataset)} sliding window samples")

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])
y_locomotion = np.array([y[0].item() for x, y in dataset])
y_terrain = np.array([y[1].item() for x, y in dataset])
y_gait = np.array([y[2].item() for x, y in dataset])

# Split data
X_train, X_test, y_lo_train, y_lo_test = train_test_split(X, y_locomotion, test_size=0.2, random_state=42)
_, _, y_tr_train, y_tr_test = train_test_split(X, y_terrain, test_size=0.2, random_state=42)
_, _, y_ga_train, y_ga_test = train_test_split(X, y_gait, test_size=0.2, random_state=42)

# Train and evaluate
meta_knn = MetaKNN()
meta_knn.train(X_train, y_lo_train, y_tr_train, y_ga_train)
results = meta_knn.evaluate(X_test, y_lo_test, y_tr_test, y_ga_test)

# Display results
print("\n=== Evaluation Results ===")
for target, acc in results.items():
    print(f"{target.capitalize()} Accuracy: {acc:.2%}")


Mounted at /content/drive
Created 9438 sliding window samples
Epoch 1/100
Training locomotion model...
Training terrain model...
Training gait phase model...
Training Accuracy: {'locomotion': 0.9964238410596027, 'terrain': 0.9943046357615895, 'gait_phase': 0.9268874172185431}
Target accuracy achieved in 1 epochs

=== Evaluation Results ===
Locomotion Accuracy: 99.79%
Terrain Accuracy: 99.58%
Gait_phase Accuracy: 90.41%


Cross subject testing (two subjects). The epoch is 1 because it is not a neural network and does not iterate through. Run time is about 51 seconds.

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Custom PyTorch Dataset
class GaitDataset(Dataset):
    def __init__(self, data, window_size=100, stride=20):
        self.window_size = window_size
        self.stride = stride

        # Features and targets
        self.feature_cols = [
            'left_hip_angle',
            'left_knee_angle',
            'right_hip_angle',
            'right_knee_angle'
        ]
        self.target_cols = [
            'locomotion_mode',
            'terrain_info',
            'gait_phase'
        ]

        # Validate columns
        missing_features = [col for col in self.feature_cols if col not in data.columns]
        missing_targets = [col for col in self.target_cols if col not in data.columns]
        if missing_features or missing_targets:
            raise ValueError(f"Missing columns: Features={missing_features}, Targets={missing_targets}")

        # Convert to numpy arrays
        self.features = data[self.feature_cols].values.astype(np.float32)
        self.labels = data[self.target_cols].values
        self.samples = self._create_windows()

    def _create_windows(self):
        windows = []
        max_start = len(self.features) - self.window_size + 1
        for i in range(0, max_start, self.stride):
            window_features = self.features[i:i+self.window_size]
            window_labels = self.labels[i+self.window_size-1]  # Last label in window
            windows.append((window_features, window_labels))
        return windows

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, labels = self.samples[idx]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long)

# Custom KNN Model in PyTorch
class PyTorchKNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            # Compute L2 distance (Euclidean)
            distances = torch.norm(self.X_train - x, dim=1)
            # Get k-nearest neighbors
            _, indices = torch.topk(distances, self.k, largest=False)
            # Vote for the most common label
            neighbor_labels = self.y_train[indices]
            unique_labels, counts = torch.unique(neighbor_labels, return_counts=True)
            pred_label = unique_labels[torch.argmax(counts)]
            predictions.append(pred_label)
        return torch.tensor(predictions)

# Meta-Learning KNN Trainer
class MetaKNN:
    def __init__(self):
        self.models = {
            'locomotion': PyTorchKNN(k=5),
            'terrain': PyTorchKNN(k=5),
            'gait_phase': PyTorchKNN(k=5)
        }
        self.encoders = {
            'locomotion': LabelEncoder(),
            'terrain': LabelEncoder(),
            'gait_phase': LabelEncoder()
        }

    def train(self, X_train, y_locomotion, y_terrain, y_gait_phase, max_epochs=100, target_accuracy=0.9):
        # Encode categorical labels
        y_locomotion_enc = self.encoders['locomotion'].fit_transform(y_locomotion)
        y_terrain_enc = self.encoders['terrain'].fit_transform(y_terrain)
        y_gait_enc = self.encoders['gait_phase'].fit_transform(y_gait_phase)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_locomotion_tensor = torch.tensor(y_locomotion_enc, dtype=torch.long)
        y_terrain_tensor = torch.tensor(y_terrain_enc, dtype=torch.long)
        y_gait_tensor = torch.tensor(y_gait_enc, dtype=torch.long)

        # Train KNN models
        for epoch in range(max_epochs):
            print(f"Epoch {epoch + 1}/{max_epochs}")

            print("Training locomotion model...")
            self.models['locomotion'].fit(X_train_tensor, y_locomotion_tensor)

            print("Training terrain model...")
            self.models['terrain'].fit(X_train_tensor, y_terrain_tensor)

            print("Training gait phase model...")
            self.models['gait_phase'].fit(X_train_tensor, y_gait_tensor)

            # Evaluate
            train_acc = self.evaluate(X_train, y_locomotion, y_terrain, y_gait_phase)
            print(f"Training Accuracy: {train_acc}")

            # Check for target accuracy
            if all(acc >= target_accuracy for acc in train_acc.values()):
                print(f"Target accuracy achieved in {epoch + 1} epochs")
                break

    def evaluate(self, X_test, y_locomotion, y_terrain, y_gait_phase):
        results = {}

        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        for target in ['locomotion', 'terrain', 'gait_phase']:
            y_true = locals()[f"y_{target}"]
            y_pred_enc = self.models[target].predict(X_test_tensor)
            y_pred = self.encoders[target].inverse_transform(y_pred_enc.numpy())

            acc = accuracy_score(y_true, y_pred)
            results[target] = acc

        return results

    def save_predictions(self, X_test, output_path):
        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        terrain_preds = self.models['terrain'].predict(X_test_tensor)
        terrain_preds_decoded = self.encoders['terrain'].inverse_transform(terrain_preds.numpy())

        # Save to Excel
        df = pd.DataFrame({'Incline Prediction': terrain_preds_decoded})
        df.to_excel(output_path, index=False)

# ===== MAIN EXECUTION =====
# Load data
data_dir = "/content/drive/MyDrive/tracedata1"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

# Create dataset
dataset = GaitDataset(full_data, window_size=100, stride=20)
print(f"Created {len(dataset)} sliding window samples")

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])
y_locomotion = np.array([y[0].item() for x, y in dataset])
y_terrain = np.array([y[1].item() for x, y in dataset])
y_gait = np.array([y[2].item() for x, y in dataset])

# Split data
X_train, X_test, y_lo_train, y_lo_test = train_test_split(X, y_locomotion, test_size=0.2, random_state=42)
_, _, y_tr_train, y_tr_test = train_test_split(X, y_terrain, test_size=0.2, random_state=42)
_, _, y_ga_train, y_ga_test = train_test_split(X, y_gait, test_size=0.2, random_state=42)

# Train and evaluate
meta_knn = MetaKNN()
meta_knn.train(X_train, y_lo_train, y_tr_train, y_ga_train)
results = meta_knn.evaluate(X_test, y_lo_test, y_tr_test, y_ga_test)

# Display results
print("\n=== Evaluation Results ===")
for target, acc in results.items():
    print(f"{target.capitalize()} Accuracy: {acc:.2%}")

# Save predictions to Excel
output_path = "/content/drive/MyDrive/incline_predictions.xlsx"
meta_knn.save_predictions(X_test, output_path)
print(f"Predictions saved to {output_path}")


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Created 18779 sliding window samples
Epoch 1/100
Training locomotion model...
Training terrain model...
Training gait phase model...
Training Accuracy: {'locomotion': 0.9874192904213539, 'terrain': 0.9829594621580243, 'gait_phase': 0.9293083937961792}
Target accuracy achieved in 1 epochs

=== Evaluation Results ===
Locomotion Accuracy: 97.82%
Terrain Accuracy: 97.02%
Gait_phase Accuracy: 89.70%
Predictions saved to /content/drive/MyDrive/incline_predictions.xlsx


Cross subject testing (3 subjects)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Custom PyTorch Dataset
class GaitDataset(Dataset):
    def __init__(self, data, window_size=100, stride=20):
        self.window_size = window_size
        self.stride = stride

        # Features and targets
        self.feature_cols = [
            'left_hip_angle',
            'left_knee_angle',
            'right_hip_angle',
            'right_knee_angle'
        ]
        self.target_cols = [
            'locomotion_mode',
            'terrain_info',
            'gait_phase'
        ]

        # Validate columns
        missing_features = [col for col in self.feature_cols if col not in data.columns]
        missing_targets = [col for col in self.target_cols if col not in data.columns]
        if missing_features or missing_targets:
            raise ValueError(f"Missing columns: Features={missing_features}, Targets={missing_targets}")

        # Convert to numpy arrays
        self.features = data[self.feature_cols].values.astype(np.float32)
        self.labels = data[self.target_cols].values
        self.samples = self._create_windows()

    def _create_windows(self):
        windows = []
        max_start = len(self.features) - self.window_size + 1
        for i in range(0, max_start, self.stride):
            window_features = self.features[i:i+self.window_size]
            window_labels = self.labels[i+self.window_size-1]  # Last label in window
            windows.append((window_features, window_labels))
        return windows

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, labels = self.samples[idx]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long)

# Custom KNN Model in PyTorch
class PyTorchKNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            # Compute L2 distance (Euclidean)
            distances = torch.norm(self.X_train - x, dim=1)
            # Get k-nearest neighbors
            _, indices = torch.topk(distances, self.k, largest=False)
            # Vote for the most common label
            neighbor_labels = self.y_train[indices]
            unique_labels, counts = torch.unique(neighbor_labels, return_counts=True)
            pred_label = unique_labels[torch.argmax(counts)]
            predictions.append(pred_label)
        return torch.tensor(predictions)

# Meta-Learning KNN Trainer
class MetaKNN:
    def __init__(self):
        self.models = {
            'locomotion': PyTorchKNN(k=5),
            'terrain': PyTorchKNN(k=5),
            'gait_phase': PyTorchKNN(k=5)
        }
        self.encoders = {
            'locomotion': LabelEncoder(),
            'terrain': LabelEncoder(),
            'gait_phase': LabelEncoder()
        }

    def train(self, X_train, y_locomotion, y_terrain, y_gait_phase, max_epochs=100, target_accuracy=0.9):
        # Encode categorical labels
        y_locomotion_enc = self.encoders['locomotion'].fit_transform(y_locomotion)
        y_terrain_enc = self.encoders['terrain'].fit_transform(y_terrain)
        y_gait_enc = self.encoders['gait_phase'].fit_transform(y_gait_phase)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_locomotion_tensor = torch.tensor(y_locomotion_enc, dtype=torch.long)
        y_terrain_tensor = torch.tensor(y_terrain_enc, dtype=torch.long)
        y_gait_tensor = torch.tensor(y_gait_enc, dtype=torch.long)

        # Train KNN models
        for epoch in range(max_epochs):
            print(f"Epoch {epoch + 1}/{max_epochs}")

            print("Training locomotion model...")
            self.models['locomotion'].fit(X_train_tensor, y_locomotion_tensor)

            print("Training terrain model...")
            self.models['terrain'].fit(X_train_tensor, y_terrain_tensor)

            print("Training gait phase model...")
            self.models['gait_phase'].fit(X_train_tensor, y_gait_tensor)

            # Evaluate
            train_acc = self.evaluate(X_train, y_locomotion, y_terrain, y_gait_phase)
            print(f"Training Accuracy: {train_acc}")

            # Check for target accuracy
            if all(acc >= target_accuracy for acc in train_acc.values()):
                print(f"Target accuracy achieved in {epoch + 1} epochs")
                break

    def evaluate(self, X_test, y_locomotion, y_terrain, y_gait_phase):
        results = {}

        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        for target in ['locomotion', 'terrain', 'gait_phase']:
            y_true = locals()[f"y_{target}"]
            y_pred_enc = self.models[target].predict(X_test_tensor)
            y_pred = self.encoders[target].inverse_transform(y_pred_enc.numpy())

            acc = accuracy_score(y_true, y_pred)
            results[target] = acc

        return results

    def save_predictions(self, X_test, output_path):
        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        terrain_preds = self.models['terrain'].predict(X_test_tensor)
        terrain_preds_decoded = self.encoders['terrain'].inverse_transform(terrain_preds.numpy())

        # Save to Excel
        df = pd.DataFrame({'Incline Prediction': terrain_preds_decoded})
        df.to_excel(output_path, index=False)

# ===== MAIN EXECUTION =====
# Load data
data_dir = "/content/drive/MyDrive/tracedata1"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

# Create dataset
dataset = GaitDataset(full_data, window_size=100, stride=20)
print(f"Created {len(dataset)} sliding window samples")

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])
y_locomotion = np.array([y[0].item() for x, y in dataset])
y_terrain = np.array([y[1].item() for x, y in dataset])
y_gait = np.array([y[2].item() for x, y in dataset])

# Split data
X_train, X_test, y_lo_train, y_lo_test = train_test_split(X, y_locomotion, test_size=0.2, random_state=42)
_, _, y_tr_train, y_tr_test = train_test_split(X, y_terrain, test_size=0.2, random_state=42)
_, _, y_ga_train, y_ga_test = train_test_split(X, y_gait, test_size=0.2, random_state=42)

# Train and evaluate
meta_knn = MetaKNN()
meta_knn.train(X_train, y_lo_train, y_tr_train, y_ga_train)
results = meta_knn.evaluate(X_test, y_lo_test, y_tr_test, y_ga_test)

# Display results
print("\n=== Evaluation Results ===")
for target, acc in results.items():
    print(f"{target.capitalize()} Accuracy: {acc:.2%}")

# Save predictions to Excel
output_path = "/content/drive/MyDrive/incline_predictions.xlsx"
meta_knn.save_predictions(X_test, output_path)
print(f"Predictions saved to {output_path}")


Mounted at /content/drive
Created 26204 sliding window samples
Epoch 1/100
Training locomotion model...
Training terrain model...
Training gait phase model...
Training Accuracy: {'locomotion': 0.9875017888660974, 'terrain': 0.9815389018747317, 'gait_phase': 0.9300195582693317}
Target accuracy achieved in 1 epochs

=== Evaluation Results ===
Locomotion Accuracy: 98.00%
Terrain Accuracy: 97.06%
Gait_phase Accuracy: 89.85%
Predictions saved to /content/drive/MyDrive/incline_predictions.xlsx


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Mount Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Custom PyTorch Dataset
class GaitDataset(Dataset):
    def __init__(self, data, window_size=100, stride=20):
        self.window_size = window_size
        self.stride = stride

        # Features and targets
        self.feature_cols = [
            'left_hip_angle',
            'left_knee_angle',
            'right_hip_angle',
            'right_knee_angle'
        ]
        self.target_cols = [
            'locomotion_mode',
            'terrain_info',
            'gait_phase'
        ]

        # Validate columns
        missing_features = [col for col in self.feature_cols if col not in data.columns]
        missing_targets = [col for col in self.target_cols if col not in data.columns]
        if missing_features or missing_targets:
            raise ValueError(f"Missing columns: Features={missing_features}, Targets={missing_targets}")

        # Convert to numpy arrays
        self.features = data[self.feature_cols].values.astype(np.float32)
        self.labels = data[self.target_cols].values
        self.samples = self._create_windows()

    def _create_windows(self):
        windows = []
        max_start = len(self.features) - self.window_size + 1
        for i in range(0, max_start, self.stride):
            window_features = self.features[i:i+self.window_size]
            window_labels = self.labels[i+self.window_size-1]  # Last label in window
            windows.append((window_features, window_labels))
        return windows

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        features, labels = self.samples[idx]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long)

# Custom KNN Model in PyTorch
class PyTorchKNN:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            # Compute L2 distance (Euclidean)
            distances = torch.norm(self.X_train - x, dim=1)
            # Get k-nearest neighbors
            _, indices = torch.topk(distances, self.k, largest=False)
            # Vote for the most common label
            neighbor_labels = self.y_train[indices]
            unique_labels, counts = torch.unique(neighbor_labels, return_counts=True)
            pred_label = unique_labels[torch.argmax(counts)]
            predictions.append(pred_label)
        return torch.tensor(predictions)

# Meta-Learning KNN Trainer
class MetaKNN:
    def __init__(self):
        self.models = {
            'locomotion': PyTorchKNN(k=5),
            'terrain': PyTorchKNN(k=5),
            'gait_phase': PyTorchKNN(k=5)
        }
        self.encoders = {
            'locomotion': LabelEncoder(),
            'terrain': LabelEncoder(),
            'gait_phase': LabelEncoder()
        }

    def train(self, X_train, y_locomotion, y_terrain, y_gait_phase, max_epochs=100, target_accuracy=0.9):
        # Encode categorical labels
        y_locomotion_enc = self.encoders['locomotion'].fit_transform(y_locomotion)
        y_terrain_enc = self.encoders['terrain'].fit_transform(y_terrain)
        y_gait_enc = self.encoders['gait_phase'].fit_transform(y_gait_phase)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_locomotion_tensor = torch.tensor(y_locomotion_enc, dtype=torch.long)
        y_terrain_tensor = torch.tensor(y_terrain_enc, dtype=torch.long)
        y_gait_tensor = torch.tensor(y_gait_enc, dtype=torch.long)

        # Train KNN models
        for epoch in range(max_epochs):
            print(f"Epoch {epoch + 1}/{max_epochs}")

            print("Training locomotion model...")
            self.models['locomotion'].fit(X_train_tensor, y_locomotion_tensor)

            print("Training terrain model...")
            self.models['terrain'].fit(X_train_tensor, y_terrain_tensor)

            print("Training gait phase model...")
            self.models['gait_phase'].fit(X_train_tensor, y_gait_tensor)

            # Evaluate
            train_acc = self.evaluate(X_train, y_locomotion, y_terrain, y_gait_phase)
            print(f"Training Accuracy: {train_acc}")

            # Check for target accuracy
            if all(acc >= target_accuracy for acc in train_acc.values()):
                print(f"Target accuracy achieved in {epoch + 1} epochs")
                break

    def evaluate(self, X_test, y_locomotion, y_terrain, y_gait_phase):
        results = {}

        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        for target in ['locomotion', 'terrain', 'gait_phase']:
            y_true = locals()[f"y_{target}"]
            y_pred_enc = self.models[target].predict(X_test_tensor)
            y_pred = self.encoders[target].inverse_transform(y_pred_enc.numpy())

            acc = accuracy_score(y_true, y_pred)
            results[target] = acc

            # Confusion Matrix
            cm = confusion_matrix(y_true, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=self.encoders[target].classes_, yticklabels=self.encoders[target].classes_)
            plt.title(f'Confusion Matrix for {target.capitalize()}')
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.show()

        return results

    def save_predictions(self, X_test, output_path):
        # Convert test data to PyTorch tensors
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

        # Predict and decode labels
        terrain_preds = self.models['terrain'].predict(X_test_tensor)
        terrain_preds_decoded = self.encoders['terrain'].inverse_transform(terrain_preds.numpy())

        # Save to Excel
        df = pd.DataFrame({'Incline Prediction': terrain_preds_decoded})
        df.to_excel(output_path, index=False)

# ===== MAIN EXECUTION =====
# Load data
data_dir = "/content/drive/MyDrive/tracedata1"
all_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
full_data = pd.concat([pd.read_csv(os.path.join(data_dir, f)) for f in all_files])

# Create dataset
dataset = GaitDataset(full_data, window_size=100, stride=20)
print(f"Created {len(dataset)} sliding window samples")

# Prepare training data
X = np.array([x.numpy().flatten() for x, y in dataset])
y_locomotion = np.array([y[0].item() for x, y in dataset])
y_terrain = np.array([y[1].item() for x, y in dataset])
y_gait = np.array([y[2].item() for x, y in dataset])

# Split data
X_train, X_test, y_lo_train, y_lo_test = train_test_split(X, y_locomotion, test_size=0.2, random_state=42)
_, _, y_tr_train, y_tr_test = train_test_split(X, y_terrain, test_size=0.2, random_state=42)
_, _, y_ga_train, y_ga_test = train_test_split(X, y_gait, test_size=0.2, random_state=42)

# Train and evaluate
meta_knn = MetaKNN()
meta_knn.train(X_train, y_lo_train, y_tr_train, y_ga_train)
results = meta_knn.evaluate(X_test, y_lo_test, y_tr_test, y_ga_test)

# Display results
print("\n=== Evaluation Results ===")
for target, acc in results.items():
    print(f"{target.capitalize()} Accuracy: {acc:.2%}")

# Save predictions to Excel
output_path = "/content/drive/MyDrive/incline_predictions.xlsx"
meta_knn.save_predictions(X_test, output_path)
print(f"Predictions saved to {output_path}")

# Plot Gait Phase and Incline Signals
plt.figure(figsize=(12, 6))
plt.plot(full_data['gait_phase'], label='Gait Phase')
plt.plot(full_data['terrain_info'], label='Incline')
plt.title('Gait Phase and Incline Signals')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Created 28554 sliding window samples
Epoch 1/100
Training locomotion model...
Training terrain model...
Training gait phase model...
